In [6]:
#!pip install prettytable
#!pip install center_tk_window
#!pip install keras
#!pip install tensorflow
#!pip install lime
#!pip install gensim
#!pip install transformers
#!pip install torch
#!pip install tensorboardX
#!pip install simpletransformers
#!pip install wandb

^C


In [43]:
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter import scrolledtext
import math
import os
import center_tk_window
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pickle
import numpy as np
import tkinter as tk
from tkinter import scrolledtext
from tkinter import ttk
import center_tk_window
from prettytable import PrettyTable

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import re
import nltk
from nltk.tokenize import word_tokenize
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

## for explainer
from lime import lime_text

## for word embedding
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers

win = tk.Tk()
#win.configure(background='black')
win.geometry("840x620")
win.configure(bg='blue')
win.title('Graphical Transformations')
win.resizable(False, False)
center_tk_window.center_on_screen(win)
font = ("Segoe Boot Semilight", 10)
font2 = ("Segoe Boot Semilight", 12)

Frame1 = tk.Frame(win, width=840, height=300, relief='sunken', bg='#0082BE')
Frame1.grid_propagate(0)
Frame1.grid(row=0)

Frame2 = tk.Frame(win, width=840, height=30, bg='#0082BE')
Frame2.grid_propagate(0)
Frame2.grid(row=1)    

Frame3 = tk.Frame(win, width=840, height=300, bg='#0082BE')
Frame3.grid_propagate(0)
Frame3.grid(row=2)

#Loading Naive Bayes for all languages

e = open('model/NB_model.pickle', 'rb')
nb_english = pickle.load(e)
e.close()
#Yoruba
y = open('model/NB_model_yoruba.pickle', 'rb')
nb_yoruba = pickle.load(y)
y.close()
#Hausa
h = open('model/NB_model_hausa.pickle', 'rb')
nb_hausa = pickle.load(h)
h.close()

#Loading LSTM
#English
from keras.models import load_model
LSTM_english = load_model('model/LSTM.h5')
#Yoruba
LSTM_yoruba = load_model('model/LSTM_yoruba.h5')
#Hausa
LSTM_hausa = load_model('model/LSTM_hausa.h5')

#Loading LSTM
#English

#Load BERT
import torch
BERT_english = torch.load('model/model')
BERT_yoruba = torch.load('model/bert_model_yoruba')
BERT_hausa = torch.load('model/bert_model_hausa')

#Selecting Transformation Type                    

def radCall():
    if radVar.get() == 1:
        tk.Label(Frame1, text="Enter your text", font=font2, bg='#0082BE', fg='white').grid(column=0, row=1, padx=(140,0), pady=(10,10))
        scrol_w = 102
        scrol_h = 23
        scr = scrolledtext.ScrolledText(Frame1, width=scrol_w, height=scrol_h, wrap=tk.WORD)
        scr.grid(column=0, row=2, columnspan=3, padx=(5,5))
        text=scrolledtext.ScrolledText(Frame3, height=18, width=102)
        text.tag_configure("center", justify='center')
        text.grid(column=0, row=3, padx=(5,5))
        def comando():
            global coord_1, coord_2, coord_3, coord_4
            result=scr.get("1.0","end")
            NB_e = nb_english.predict([result])
            NB_e_proba = nb_english.predict_proba([result])
            if NB_e == ['BUSINESS']:
                predictedNBEnglish = 'BUSINESS'
            elif NB_e == ['ENTERTAINMENT']:
                predictedNBEnglish = 'ENTERTAINMENT'
            elif NB_e == ['POLITICS']:
                predictedNBEnglish = 'POLITICS'
            elif NB_e == ['SCIENCE&TECH']:
                predictedNBEnglish = 'SCIENCE&TECH'
    #LSTM
            MAX_NB_WORDS = 50000
    # Max number of words in each complaint.
            MAX_SEQUENCE_LENGTH = 250
            # This is fixed.
            EMBEDDING_DIM = 100
            tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
            seq = tokenizer.texts_to_sequences([result])
            padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
            pred = LSTM_english.predict(padded)
            labels = ['BUSINESS', 'ENTERTAINMENT', 'POLITICS', 'SCIENCE&TECH']
            predictedLSTMEnglish = labels[np.argmax(pred)]
    
    #BERT
            bert, raw_outputs = BERT_english.predict([result])
            if bert == [0]:
                predictedBERT = 'BUSINESS'
            elif bert == [1]:
                predictedBERT = 'ENTERTAINMENT'
            elif bert == [2]:
                predictedBERT = 'POLITICS'
            elif bert == [3]:
                predictedBERT = 'SCIENCE&TECH'
    #'BUSINESS', 'ENTERTAINMENT', 'POLITICS', 'SCIENCE&TECH'
    #PrettyTable
            x=PrettyTable()
            x.field_names = ["", "Classification", "Confidence Intervals"]
            x.add_row(["Naive Bayes", predictedNBEnglish, NB_e_proba])
            x.add_row(["Long Short Term Memory (LSTM)", predictedLSTMEnglish, np.argmax(pred)])
            x.add_row(["BERT Transformer", predictedBERT, raw_outputs])

            text.insert(INSERT,x)#Inserting table in text widget
            #all_4points_text = "Naive Bayes: " + str(predictedNBEnglish) + "\n\n" + "LSTM: " + str(predictedLSTMEnglish) + "\n\n" + "BERT Transformer : " + str(predictedBERT) + "\n\n" + "....................................... \n\n"
            #text.insert(tk.END, all_4points_text)
            
        btnRead=tk.Button(Frame2, height=1, width=15, text="Classify", command=comando)
        btnRead.grid(column=0, row=0, padx=(370,0), pady=(3,0))
        
        
    elif radVar.get() == 2: 
        tk.Label(Frame1, text="Enter your text", font=font2, bg='#0082BE', fg='white').grid(column=0, row=1, padx=(140,0), pady=(10,10))
        scrol_w = 102
        scrol_h = 23
        scr = scrolledtext.ScrolledText(Frame1, width=scrol_w, height=scrol_h, wrap=tk.WORD)
        scr.grid(column=0, row=2, columnspan=3, padx=(5,5))
        text=scrolledtext.ScrolledText(Frame3, height=18, width=102)
        text.grid(column=0, row=3, padx=(5,5))
        def comando():
            global coord_1, coord_2, coord_3, coord_4
            result=scr.get("1.0","end")
            NB_y = nb_yoruba.predict([result])
            NB_y_proba = nb_yoruba.predict_proba([result])
            if NB_y == ['entertainment']:
                predictedNB_y = 'Entertainment'
            elif NB_y == ['health']:
                predictedNB_y = 'health'
            elif NB_y == ['politics']:
                predictedNB_y = 'politics'
            elif NB_y == ['sport']:
                predictedNB_y = 'sport'
            elif NB_y == ['world']:
                predictedNB_y = 'world'
    #LSTM
            MAX_NB_WORDS = 50000
    # Max number of words in each complaint.
            MAX_SEQUENCE_LENGTH = 250
    # This is fixed.
            EMBEDDING_DIM = 100
            tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
            seq = tokenizer.texts_to_sequences([result])
            padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
            pred = LSTM_yoruba.predict(padded)
            labels = ['entertainment', 'health', 'politics', 'sport', 'world']
            predictedLSTMYoruba = labels[np.argmax(pred)]
#BERT
            bert, raw_outputs = BERT_yoruba.predict([result])
            if bert == [0]:
                predictedBERT = 'Entertainment'
            elif bert == [1]:
                predictedBERT = 'Health'
            elif bert == [2]:
                predictedBERT = 'Politics'
            elif bert == [3]:
                predictedBERT = 'Sport'
            elif bert == [4]:
                predictedBERT = 'World'
    #'BUSINESS', 'ENTERTAINMENT', 'POLITICS', 'SCIENCE&TECH'
    #PrettyTable
            x=PrettyTable()
            x.field_names = ["", "Classification", "Confidence Levels"]
            x.add_row(["Naive Bayes", predictedNB_y, NB_y_proba])
            x.add_row(["Long Short Term Memory (LSTM)", predictedLSTMYoruba, np.argmax(pred)])
            x.add_row(["BERT Transformer", predictedBERT, raw_outputs])

            text.insert(INSERT,x)#Inserting table in text widget
            #all_4points_text = "Naive Bayes: " + str(predictedNB_y) + "\n\n" + "LSTM: " + str(predictedLSTMYoruba) + "\n\n" + "....................................... \n\n"
            #text.insert(tk.END, all_4points_text)

            
        btnRead=tk.Button(Frame2, height=1, width=15, text="Classify", command=comando)
        btnRead.grid(column=0, row=0, padx=(370,0), pady=(3,0))
        #tk.Button(win, text="Compute", command=comando, width=20).grid(row=5, column=0, padx=(320,0), pady=5)

    elif radVar.get() == 3: 
        tk.Label(Frame1, text="Enter your text", font=font2, bg='#0082BE', fg='white').grid(column=0, row=1, padx=(140,0), pady=(10,10))
        scrol_w = 102
        scrol_h = 23
        scr = scrolledtext.ScrolledText(Frame1, width=scrol_w, height=scrol_h, wrap=tk.WORD)
        scr.grid(column=0, row=2, columnspan=3, padx=(5,5))
        text=scrolledtext.ScrolledText(Frame3, height=18, width=102)
        text.grid(column=0, row=3, padx=(5,5))
        def comando():
            global coord_1, coord_2, coord_3, coord_4
            result=scr.get("1.0","end")
            NB_h = nb_hausa.predict([result])
            NB_h_proba = nb_hausa.predict([result])
            if NB_h == ['entertainment']:
                predictedNB_h = 'Entertainment'
            elif NB_h == ['Health']:
                predictedNB_h = 'Health'
            elif NB_h == ['Politics']:
                predictedNB_h = 'Politics'
            elif NB_h == ['World']:
                predictedNB_h = 'World'
    #LSTM
            MAX_NB_WORDS = 50000
    # Max number of words in each complaint.
            MAX_SEQUENCE_LENGTH = 250
    # This is fixed.
            EMBEDDING_DIM = 100
            tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
            seq = tokenizer.texts_to_sequences([result])
            padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
            pred = LSTM_hausa.predict(padded)
            labels = ['Health', 'Politics', 'World']
            predictedLSTMHausa = labels[np.argmax(pred)]

#BERT'Health', 'Politics', 'World'
            bert, raw_outputs = BERT_yoruba.predict([result])
            if bert == [0]:
                predictedBERT = 'Health'
            elif bert == [1]:
                predictedBERT = 'Politics'
            elif bert == [2]:
                predictedBERT = 'World'
    #'BUSINESS', 'ENTERTAINMENT', 'POLITICS', 'SCIENCE&TECH'
    #PrettyTable
            x=PrettyTable()
            x.field_names = ["", "Classification", "Confidence Levels"]
            x.add_row(["Naive Bayes", predictedNB_h, NB_h_proba])
            x.add_row(["Long Short Term Memory (LSTM)", predictedLSTMHausa, np.argmax(pred)])
            x.add_row(["BERT Transformer", predictedBERT, raw_ouputs])

            text.insert(INSERT,x)#Inserting table in text widget

#            all_4points_text = "Naive Bayes: " + str(predictedNB_h) + "\n\n" + "LSTM: " + str(predictedLSTMHausa) + "\n\n" + "....................................... \n\n"
 #           text.insert(tk.END, all_4points_text)

            
        btnRead=tk.Button(Frame2, height=1, width=15, text="Classify", command=comando)
        btnRead.grid(column=0, row=0, padx=(370,0), pady=(3,0))

#def destroy_frame():
    
#tk.Button(win, text='Reload Graph', width=10, command=destroy_frame).grid(row=5, column=1)
        
word1 = 'Select the Language: '
Label1 = tk.Label(Frame1, text=word1, fg = 'white', font=font, bg='#0082BE')
Label1.grid(row=0, column=0, sticky=tk.W, pady=2, padx=(190,0))

radVar = tk.IntVar()
English = tk.Radiobutton(Frame1, text='English', variable = radVar, value=1, fg='white',
                            activeforeground='#0082BE', selectcolor='#0082BE', command=radCall, bg='#0082BE', activebackground='#0082BE')
English.grid(column=0, row=0, sticky=tk.W, padx=(370, 0))
Yoruba = tk.Radiobutton(Frame1, text='Yoruba', variable = radVar, value=2, fg='white',
                      activeforeground='#0082BE', selectcolor='#0082BE', command = radCall, bg='#0082BE', activebackground='#0082BE')
Yoruba.grid(column=0, row=0, sticky=tk.W, padx=(450,0))
Hausa = tk.Radiobutton(Frame1, text='Hausa', variable = radVar, value=3, fg='white',
                      activeforeground='#0082BE', selectcolor='#0082BE', command = radCall, bg='#0082BE', activebackground='#0082BE')
Hausa.grid(column=0, row=0, sticky=tk.W, padx=(530,0))
win.update()
win.mainloop()

Phase 1 Done
Phase 1 Done
Phase 1 Done
Phase 1 Done
Phase 1 Done
Phase 1 Done
